In [ ]:
# #merge data
# import os
# import glob
# import pandas as pd

# path = 'Tuber_HIV_data/'
# os.chdir(path)

# all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]

# combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# combined_csv.to_csv( "tuberculasis_HIV.csv", index=False, encoding='utf-8-sig')

In [ ]:
#df.to_csv (r'C:\workspace\DataForRegressor.csv', index = False, header=True)
import pandas as pd 
data1= pd.read_csv("data_Tuberculosis_HIV1.csv")
data1['comparison']='tuberculosis_HIV'
data2= pd.read_csv("data_Tuberculosis_Covid19.csv")
data2['comparison']='tuberculosis_Covid19'
data3= pd.read_csv("dat_HIV1_COVID19.csv")
data3['comparison']='HIV_COVID19'
df = pd.concat([data1,data2])
df.head(5)

In [ ]:
df_data=df.drop(['mean','STD','signalToNoise','comparison'], axis=1)
data=df_data.dropna()
data.head(5)

In [ ]:
import numpy as np
!pip install xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import time
def getRegressor(df,estimators,maxDepth,randomState):
    from sklearn.preprocessing import StandardScaler
    from sklearn.neural_network import MLPClassifier
    scaler = StandardScaler()  
    X=df.copy()
    y=df['global_score']
    X_trainBase, X_testBase, y_train, y_test = train_test_split(X, y)
    X_train=X_trainBase.drop(['chunk_or1','chunk_or2','global_score'], axis=1)
    X_train=X_train.astype('int32')
    X_train =scaler.fit_transform(X_train)
    #X_train = scaler.transform(X_train)
    X_test=X_testBase.drop(['chunk_or1','chunk_or2','global_score'], axis=1)
    X_test=X_test.astype('int32')
    X_test = scaler.transform(X_test)
    model= XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=5,
             min_child_weight=4, monotone_constraints='()',
             n_estimators=600, n_jobs=4, nthread=4, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, silent=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None).fit(X_train, y_train)
    start = time.time()
    result = model.score(X_test,y_test)
    end = time.time()
    print("Score:"+ str(result))
    print("Time for executing model")
    print(end - start)
    print(" in seconds")
    return model,X_test,X_testBase,X_trainBase,X_train


In [ ]:
!pip install xgboost
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
model = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07, 0.1], #so called `eta` value
              'max_depth': [5, 6, 7,8],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [200,300,500,600,700]}

grid = GridSearchCV(model,parameters, cv=2,n_jobs = 5,
                        verbose=True)
grid.fit(X_train, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=5,
             min_child_weight=4, monotone_constraints='()',
             n_estimators=600, n_jobs=4, nthread=4, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, silent=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
regression_results(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
#plt.rcParams['agg.path.chunksize'] = 10000
test=pd.DataFrame(y_test)
test['pred']=y_pred
test['rank_distance']=X_testBase['rank_distance']
dt=test[0:100]
#dt.head()
dt.plot(x='rank_distance', y=['global_score','pred'], figsize=(10,5), grid=True)
# plt.scatter(dt['index'], y_test, color = "red")
# plt.plot(dt['index'], y_pred, color = "green")
# plt.title("Test")
# plt.xlabel("index")
# plt.ylabel("value")
# plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression({'copy_X': True, 'fit_intercept': False, 'normalize': True}).fit(X_train, y_train)
y_pred=reg.predict(X_test)
regression_results(y_test, y_pred)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(1,15)],
              'activation': ['relu'],
              'solver': ['adam'],
              'learning_rate': ['constant'],
              'learning_rate_init': [0.001],
              'power_t': [0.5],
              'alpha': [0.0001],
              'max_iter': [1000],
              'early_stopping': [False],
              'warm_start': [False]}
grid = GridSearchCV(mlp, param_grid=param_grid, verbose=True)
grid.fit(X_train, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

regr = MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=11, learning_rate='constant',
             learning_rate_init=0.001, max_iter=1000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False).fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [ ]:
 regression_results(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
#plt.rcParams['agg.path.chunksize'] = 10000
test=pd.DataFrame(y_test)
test['pred']=y_pred
test['rank_distance']=X_testBase['rank_distance']
dt=test[0:100]
#dt.head()
dt.plot(x='rank_distance', y=['global_score','pred'], figsize=(10,5), grid=True)

In [ ]:
!pip install ml_metrics
import sklearn.metrics as metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from ml_metrics import rmse
import numpy as np

def regression_results(y_true, y_pred):# Regression metrics
    explained_variance=explained_variance_score(y_true, y_pred)
    mae=mean_absolute_error(y_true, y_pred) 
    mse=mean_squared_error(y_true, y_pred) 
    r2=r2_score(y_true, y_pred)
    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mae,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [ ]:
model,X_test,X_testBase,X_trainBase,X_train=getRegressor(data,200,20,0)

In [ ]:
#verificate result of train and test
X_testBase['globalAlign_pred']=model.predict(X_test)
X_trainBase['globalAlign_pred']=model.predict(X_train)
prediction_test = pd.DataFrame(X_testBase, columns=['globalAlign_pred'])
prediction_train = pd.DataFrame(X_trainBase, columns=['globalAlign_pred'])
baseline_test = pd.DataFrame(X_testBase, columns=['global_score'])
baseline_train = pd.DataFrame(X_trainBase, columns=['global_score'])

In [ ]:
# def calcMeanStdAndSignalToNoise(prediction,column):
#     prediction['mean'] = prediction[column].rolling(window=3).mean()
#     prediction['STD'] = prediction[column].rolling(window=3).std()
#     prediction['signalToNoise']=prediction['mean']/prediction['STD']
#     prediction=prediction.replace([np.inf, -np.inf], np.nan)
#     prediction=prediction.fillna(0)
#     return prediction


In [ ]:
# pred_train=calcMeanStdAndSignalToNoise(prediction_train)
# pred_test=calcMeanStdAndSignalToNoise(prediction_test)
# base_train=calcMeanStdAndSignalToNoise(baseline_train)
# base_test=calcMeanStdAndSignalToNoise(baseline_test)

In [ ]:
# prediction['mean'] = data['globalAlign_pred'].rolling(window=3).mean()
# prediction['STD'] = data['globalAlign_pred'].rolling(window=3).std()
# prediction['signalToNoise']=data['mean']/data['STD']
# prediction=prediction.replace([np.inf, -np.inf], np.nan)
# prediction=prediction.fillna(0)
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# prediction=scaler.fit_transform(prediction)
# baseline['mean'] =baseline['global_score'].rolling(window=3).mean()
# baseline['STD'] = baseline['global_score'].rolling(window=3).std()
# baseline['signalToNoise']=baseline['mean']/baseline['STD']
# baseline=baseline.replace([np.inf, -np.inf], np.nan)
# baseline=baseline.fillna(0)
# baseline=scaler.fit_transform(baseline)

In [ ]:
def map_data_to_clusters(X_train,X_test):
    from sklearn.cluster import KMeans
    cluster = KMeans(n_clusters=5, init='k-means++', random_state=0)
    cluster.fit_predict(X_train)
    return cluster.predict(X_test),cluster

In [ ]:
def nb_cluster(labels_true, labels_pred,X_test):
    import sklearn
    ami_score=sklearn.metrics.adjusted_mutual_info_score(labels_true, labels_pred)
    compl_score=sklearn.metrics.completeness_score(labels_true, labels_pred)
    adj_rand_score=sklearn.metrics.adjusted_rand_score(labels_true, labels_pred)
    calinski_harabasz_score=sklearn.metrics.calinski_harabasz_score(X_test, labels_pred)
    davies_bouldin_score=sklearn.metrics.davies_bouldin_score(X_test, labels_pred)
    fowlkes_mallows_score=sklearn.metrics.fowlkes_mallows_score(labels_true, labels_pred)
    homogenity_score=sklearn.metrics.homogeneity_score(labels_true, labels_pred)
    mutual_info_score=sklearn.metrics.mutual_info_score(labels_true, labels_pred)
    silhouette_score=sklearn.metrics.silhouette_score(X_test, labels_pred)
    print("**************************")
    print("ami_score: {}".format(ami_score))
    print("compl_score: {}".format(compl_score))
    print("adj_rand_score: {}".format(adj_rand_score))
    print("calinski_harabasz_score: {}".format(calinski_harabasz_score))
    print("davies_bouldin_score: {}".format(davies_bouldin_score))
    print("fowlkes_mallows_score: {}".format(fowlkes_mallows_score))
    print("homogenity_score: {}".format(homogenity_score))
    print("mutual_info_score: {}".format(mutual_info_score))
    print("silhouette_score: {}".format(silhouette_score))

In [ ]:
prediction_train

In [ ]:
pred_labels,cluster_pred=map_data_to_clusters(prediction_train,prediction_test)
baseline_labels,cluster_base=map_data_to_clusters(baseline_train,baseline_test)
prediction_test = pd.DataFrame(X_testBase, columns=['globalAlign_pred'])
prediction_train = pd.DataFrame(X_trainBase, columns=['globalAlign_pred'])
baseline_test = pd.DataFrame(X_testBase, columns=['global_score'])
baseline_train = pd.DataFrame(X_trainBase, columns=['global_score'])


In [ ]:
#result analysis-test with hiv/covid

In [ ]:
from sklearn.preprocessing import StandardScaler
df=data3.copy()
scaler = StandardScaler()  
X_hc=data3.copy()
y_hc=df['global_score']
X_test_hc=data3.drop(['chunk_or1','chunk_or2','global_score','mean','STD','signalToNoise','comparison'], axis=1)
X_test_hc=X_test_hc.astype('int32')
X_test_hc_scaled =scaler.fit_transform(X_test_hc)

In [ ]:
#X_test_hc['globalAlign_pred']
cluster_data = pd.DataFrame(model.predict(X_test_hc_scaled), columns=['globalAlign_pred'], index=X_test_hc.index)


In [ ]:
X_test_hc['globalAlign_pred']=model.predict(X_test_hc_scaled)
X_test_hc['clusters']=cluster_pred.predict(cluster_data)

In [ ]:
len(data3)

In [ ]:
len(X_test_hc.index)

In [ ]:
X_test_hc['chunk_or1']=data3['chunk_or1']
X_test_hc['chunk_or2']=data3['chunk_or2']
X_test_hc['comparison']=data3['comparison']

In [ ]:
#gfg_csv_data = df.to_csv('result_HIV_COVID.csv', index = True) 

In [ ]:
#testM=X_test_hc[X_test_hc['chunk_or2'].str.contains('TGTTAAAGCCACTTGCGAATTTTGTGGCACTGAGAATTTGACTAAAGAAGGTGCCACTACTTGTGGTTACTTACCCCAAAATGCTGTTGTTAAAATTT', regex=False)]
#testM[(testM.clusters==1)][['clusters','chunk_or1','chunk_or2','globalAlign_pred']]
print(X_test_hc['chunk_or1'][166])
print(X_test_hc['chunk_or2'][166])
# testA=testM[testM['chunk_or1'].str.contains('', regex=False)]
# testA[['clusters','chunk_or1','chunk_or2','globalAlign_pred']]
# testA[(testA.clusters==1)][['clusters','chunk_or1','chunk_or2','globalAlign_pred']]

In [ ]:
testM=X_test_hc[X_test_hc['chunk_or1'].str.contains('CATCCGGAGTACTTCAAGAAC', regex=False)]
testM=testM[(testM.clusters==1)][['clusters','chunk_or1','chunk_or2','globalAlign_pred']]
testA=testM[testM['chunk_or2'].str.contains('CTAAAGCTATTAA', regex=False)]
#testA
testA

In [ ]:
X_test_hc[(X_test_hc.clusters==1)][['chunk_or1','chunk_or2','globalAlign_pred']][10:20]

In [ ]:
X_test_hc['chunk_or2'][1].find('TCTCTCTGGTTAGACCAGATCT')  
# start = 100 
# end = 1000
# any_string.index('substring', start, end)

In [ ]:
len(X_test_hc[(X_test_hc.clusters==1)])

In [ ]:
d['chunk_or2']

In [ ]:
'GATGGTGGCAGTTTGTATGTAAATAAACATGCATTCCACACACCAGCTTTTGATAAAAGT'in d['chunk_or2'][8]

In [ ]:
d=X_test_hc[(X_test_hc.clusters==1)]
d[['globalAlign_pred']].max()


In [ ]:
nb_cluster(baseline_labels, pred_labels,prediction_test)

In [ ]:
#Accuracy
labelsAB=map_data_to_clusters(baseline)
labelsAF=map_data_to_clusters(data)
#print(cluster_ab_1['data_index'])
'''here manually decid ewhich cluster is with P example and which is with negative examples'''
from sklearn.metrics.cluster import adjusted_rand_score
score=adjusted_rand_score(labelsAB,labelsAF)
print(score)
NBClust(labelsAB,labelsAF,X_test)
NBClust(labels_true, labels_pred,X_test)


In [ ]:
!pip install pyeasyga
import time
from pyeasyga import pyeasyga
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans

In [ ]:
def fitness(individual, data):
    temp=0;
    cNum=1
    for x in data:
        if temp < x['score']:
            temp=x['score']
            cNum=x['clusterNum']
    return cNum

In [ ]:
X=data
#print(X.head(5))
start = time.time()
#KMeans++
arr=[];
for i in range(2,20):
    cluster_labels = KMeans(n_clusters=i, init='k-means++').fit_predict(X)
    arr.append({'clusterNum': i , 'score': silhouette_score(X, cluster_labels)})
    print({'clusterNum': i , 'score': silhouette_score(X, cluster_labels)})
#initialise the GA with data
ga = pyeasyga.GeneticAlgorithm(arr)
ga.fitness_function = fitness               # set the GA's fitness function
ga.run()                                    # run the GA
print(ga.best_individual())
end = time.time()
print("Time for executing Kmeans++ with GA")
print(end - start)
print(" in seconds")

In [ ]:
def NBClust(labels_true, labels_pred,X_test):
    import sklearn
    ami_score=sklearn.metrics.adjusted_mutual_info_score(labels_true, labels_pred)
    compl_score=sklearn.metrics.completeness_score(labels_true, labels_pred)
    adj_rand_score=sklearn.metrics.adjusted_rand_score(labels_true, labels_pred)
    calinski_harabasz_score=sklearn.metrics.calinski_harabasz_score(X_test, labels_pred)
    davies_bouldin_score=sklearn.metrics.davies_bouldin_score(X_test, labels_pred)
    fowlkes_mallows_score=sklearn.metrics.fowlkes_mallows_score(labels_true, labels_pred)
    homogenity_score=sklearn.metrics.homogeneity_score(labels_true, labels_pred)
    mutual_info_score=sklearn.metrics.mutual_info_score(labels_true, labels_pred)
    silhouette_score=sklearn.metrics.silhouette_score(X_test, labels_pred)
    print("**************************")
    print("ami_score: {}".format(ami_score))
    print("compl_score: {}".format(compl_score))
    print("adj_rand_score: {}".format(adj_rand_score))
    print("calinski_harabasz_score: {}".format(calinski_harabasz_score))
    print("davies_bouldin_score: {}".format(davies_bouldin_score))
    print("fowlkes_mallows_score: {}".format(fowlkes_mallows_score))
    print("homogenity_score: {}".format(homogenity_score))
    print("mutual_info_score: {}".format(mutual_info_score))
    print("м: {}".format(silhouette_score))